<a href="https://colab.research.google.com/github/jacobmillerforever/ECON_506/blob/main/506_Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction & Setup

In [ ]:
!pip install fredapi
!pip install investpy
!pip install ta
!pip install keras_tuner

In [ ]:
import tensorflow as tf
print(f"GPUs available: {tf.config.list_physical_devices('GPU')}")
print(f"Built with CUDA: {tf.test.is_built_with_cuda()}")

In [ ]:
import pandas as pd
import yfinance as yf
import datetime as dt
from fredapi import Fred
import investpy
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.model_selection import TimeSeriesSplit
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.impute import SimpleImputer
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv1D, BatchNormalization, Activation, Add, MaxPooling1D, GlobalAveragePooling1D, Dense, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers import LSTM, Dropout
import keras_tuner as kt
from tensorflow.keras.optimizers import Adam
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import precision_recall_curve
import matplotlib.pyplot as plt
import seaborn as sns
import ta

# Data Collection & Preparation

## Ticker Data from yfinance

In [ ]:
def get_ticker_data(ticker_dict, start_date, end_date):
    """
    Fetches data for multiple tickers and creates a DataFrame for each with
    single-index columns named as Ticker_ColumnName (e.g., SPY_Close)

    Parameters:
    -----------
    ticker_dict : dict
        Dictionary with display names as keys and ticker symbols as values
    start_date : str
        Start date in format 'YYYY-MM-DD'
    end_date : str
        End date in format 'YYYY-MM-DD'

    Returns:
    --------
    dict
        Dictionary with display names as keys and their respective DataFrames as values
    """
    ticker_dataframes = {}

    for display_name, ticker_symbol in ticker_dict.items():
        # Fetch data for current ticker
        data = yf.download(ticker_symbol, start=start_date, end=end_date, progress=False)

        # Handle multi-index columns if present
        if isinstance(data.columns, pd.MultiIndex):
            # Flatten the multi-index columns to single index
            data.columns = [f"{ticker_symbol}_{col[0]}" for col in data.columns]
        else:
            # If not multi-index, still rename columns to match pattern
            data.columns = [f"{ticker_symbol}_{col}" for col in data.columns]

        # Store the DataFrame in the dictionary with display name as key
        ticker_dataframes[display_name] = data

    return ticker_dataframes

tickers = {
    # Global Indices
    'Nikkei 225 (Japan)': '^N225',
    'Hang Seng (Hong Kong)': '^HSI',
    'SSE Composite (China)': '000001.SS',
    'ASX 200 (Australia)': '^AXJO',
    'DAX (Germany)': '^GDAXI',
    'FTSE 100 (UK)': '^FTSE',
    'CAC 40 (France)': '^FCHI',
    'Euro Stoxx 50 (EU)': '^STOXX50E',
    'SPY (US)': 'SPY',


    # Volatility Indices
    'VIX (US)': '^VIX',
    #'VIX Brazil': '^VXEWZ',
    #'DAX Volatility': '^VDAX',

    # Currency Pairs
    'US Dollar Index': 'DX-Y.NYB',
    #'EUR/USD': 'EURUSD=X',
    #'JPY/USD': 'JPY=X',
    #'CNY/USD': 'CNY=X',

    # Commodities
    #'Gold': 'GC=F',
    #'Crude Oil': 'CL=F',
    #'Silver': 'SI=F',
    #'Corn': 'ZC=F',
    #'Copper': 'HG=F'
}

start_date = '2000-01-01'
end_date = dt.datetime.now().strftime('%Y-%m-%d')

# Get individual DataFrames for each ticker
ticker_data = get_ticker_data(tickers, start_date, end_date)

# Display the first few rows and column names for each DataFrame
for display_name, df in ticker_data.items():
    print(f"\n{display_name} DataFrame:")
    print(f"Column names: {df.columns.tolist()}")
    print(df.head())

## Economic Indicators from FRED API

In [ ]:
def get_fred_data(api_key, series_list, start_date='2000-01-01', end_date=None):
    """
    Fetches data for multiple FRED series at the highest available frequency

    Parameters:
    -----------
    api_key : str
        Your FRED API key
    series_list : list
        List of FRED series IDs as strings
    start_date : str, optional
        Start date in format 'YYYY-MM-DD', defaults to '2000-01-01'
    end_date : str, optional
        End date in format 'YYYY-MM-DD', defaults to current date

    Returns:
    --------
    dict
        Dictionary with series IDs as keys and their respective DataFrames as values
    dict
        Dictionary with series IDs as keys and the frequency used as values
    """
    # Initialize FRED API connection
    fred = Fred(api_key=api_key)

    # Set end date to current date if not provided
    if end_date is None:
        end_date = dt.datetime.now().strftime('%Y-%m-%d')

    # Convert start and end dates to datetime objects
    start_dt = dt.datetime.strptime(start_date, '%Y-%m-%d')
    end_dt = dt.datetime.strptime(end_date, '%Y-%m-%d')

    # Initialize dictionaries to store DataFrames and frequencies
    fred_dataframes = {}
    fred_frequencies = {}

    # Frequency hierarchy from highest to lowest resolution
    # Not all series support all frequencies
    frequency_hierarchy = ['d', 'w', 'bw', 'm', 'q', 'sa', 'a']

    # Process each series ID
    for series_id in series_list:
        # Try frequencies in order from highest to lowest resolution
        for freq in frequency_hierarchy:
            try:
                # Get data for current series with current frequency
                data = fred.get_series(series_id, start_dt, end_dt, frequency=freq)

                # If successful and data is not empty, convert to DataFrame
                if not data.empty:
                    # Convert Series to DataFrame
                    df = pd.DataFrame(data)
                    df.columns = [f"{series_id}_value"]

                    # Add to dictionaries
                    fred_dataframes[series_id] = df
                    fred_frequencies[series_id] = freq

                    print(f"Successfully fetched data for {series_id} with frequency '{freq}'")
                    # Break out of frequency loop once we've found a working frequency
                    break
                else:
                    print(f"No data found for {series_id} with frequency '{freq}'")
            except Exception as e:
                # If this frequency doesn't work, try the next one
                print(f"Could not fetch {series_id} with frequency '{freq}': {str(e)}")

        # Check if we were able to fetch this series with any frequency
        if series_id not in fred_dataframes:
            print(f"Failed to fetch data for {series_id} with any available frequency")

    return fred_dataframes, fred_frequencies

from google.colab import userdata
fred_api = '8b000b950d5841b5b7e35ebbcacedaea'

fred_series = [
    'DFF',           # Federal Funds Rate
    'T10Y2Y',        # 10-Year minus 2-Year Treasury Spread
    'CPIAUCSL',      # Consumer Price Index
    'UNRATE',        # Unemployment Rate
    'STLFSI',        # St. Louis Fed Financial Stress Index
    'M2SL',          # M2 Money Supply
    'USSLIND',       # US Leading Index
    'BAMLH0A0HYM2',  # High Yield Spread
    'GS5',           # 5-Year Treasury Rate
    'GS30',          # 30-Year Treasury Rate
    'BAMLC0A0CM'     # Corporate Bond Spread
]

fred_data = get_fred_data(fred_api, fred_series)

In [ ]:
fred_data

## Calendar Dates from investing.com

In [ ]:
calendar_df = investpy.economic_calendar(
      from_date='01/01/2000',
      to_date='31/12/2025',
      countries=['united states'],
      categories=None,
      importances=['high']
)

calendar_df = calendar_df[~calendar_df['importance'].isna()].reset_index(drop=True)
calendar_df.tail()


In [ ]:
calendar_df['event'].unique()

In [ ]:
calendar_df[calendar_df['event'] == 'FOMC Economic Projections']

# Exploratory Data Analysis

## Indices EDA

In [ ]:
def eda_indices_dict(ticker_data_dict):
    """
    Perform EDA on dictionary of DataFrame indices from yfinance

    Parameters:
    -----------
    ticker_data_dict : dict
        Dictionary with ticker symbols as keys and their DataFrames as values
    """
    print("=== EDA for Market Indices ===\n")

    # Summary statistics for each index
    for display_name, df in ticker_data_dict.items():
        print(f"\n--- {display_name} ---")
        print(f"Data range: {df.index.min().date()} to {df.index.max().date()}")
        print(f"Number of trading days: {len(df)}")

        # Handle missing data
        missing_data = df.isnull().sum()
        if missing_data.any():
            print("\nMissing values:")
            print(missing_data[missing_data > 0])

        # Calculate returns
        close_col = [col for col in df.columns if 'Close' in col][0]
        returns = df[close_col].pct_change()

        # Summary statistics for close prices
        print(f"\nClose price statistics:")
        print(f"Mean: {df[close_col].mean():.2f}")
        print(f"Std Dev: {df[close_col].std():.2f}")
        print(f"Min: {df[close_col].min():.2f}")
        print(f"Max: {df[close_col].max():.2f}")

        # Return statistics
        print(f"\nDaily return statistics:")
        print(f"Mean daily return: {returns.mean():.4%}")
        print(f"Std dev of returns: {returns.std():.4%}")
        print(f"Sharpe ratio (annualized): {(returns.mean() / returns.std() * np.sqrt(252)):.2f}")
        print(f"Skewness: {returns.skew():.2f}")
        print(f"Kurtosis: {returns.kurtosis():.2f}")

        # Plot closing prices and returns
        fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8))

        # Price chart
        ax1.plot(df.index, df[close_col])
        ax1.set_title(f"{display_name} - Closing Prices")
        ax1.set_ylabel("Price")
        ax1.grid(True, alpha=0.3)

        # Returns histogram
        ax2.hist(returns.dropna(), bins=100, alpha=0.75, color='blue', edgecolor='black')
        ax2.set_title(f"{display_name} - Return Distribution")
        ax2.set_xlabel("Daily Returns")
        ax2.set_ylabel("Frequency")
        ax2.axvline(x=0, color='red', linestyle='--', alpha=0.7)

        plt.tight_layout()
        plt.show()

    # Correlation analysis between indices
    print("\n=== Correlation Analysis ===")
    close_prices_dict = {}
    for display_name, df in ticker_data_dict.items():
        close_col = [col for col in df.columns if 'Close' in col][0]
        close_prices_dict[display_name] = df[close_col]

    close_prices_df = pd.DataFrame(close_prices_dict)
    correlation_matrix = close_prices_df.pct_change().corr()

    plt.figure(figsize=(12, 10))
    sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, vmin=-1, vmax=1)
    plt.title("Correlation Matrix of Daily Returns")
    plt.tight_layout()
    plt.show()


# Example usage:
eda_indices_dict(ticker_data)


## FRED EDA

In [ ]:
def eda_fred_data(fred_data_tuple):
    """
    Perform EDA on FRED API data

    Parameters:
    -----------
    fred_data_tuple : tuple
        Tuple containing (dataframes_dict, frequencies_dict)
    """
    dataframes_dict, frequencies_dict = fred_data_tuple

    print("=== EDA for FRED Economic Indicators ===\n")

    # Summary for each FRED series
    for series_id, df in dataframes_dict.items():
        frequency = frequencies_dict[series_id]
        print(f"\n--- {series_id} (Frequency: {frequency}) ---")
        print(f"Data range: {df.index.min().date()} to {df.index.max().date()}")
        print(f"Number of observations: {len(df)}")

        # Handle missing data
        missing_data = df.isnull().sum()
        if missing_data.any():
            print("\nMissing values:")
            print(missing_data[missing_data > 0])

        # Summary statistics
        value_col = df.columns[0]
        print(f"\nSummary statistics:")
        print(f"Mean: {df[value_col].mean():.2f}")
        print(f"Std Dev: {df[value_col].std():.2f}")
        print(f"Min: {df[value_col].min():.2f}")
        print(f"Max: {df[value_col].max():.2f}")

        # Calculate percent change based on frequency
        if frequency == 'd':
            pct_change = df[value_col].pct_change(fill_method=None)
            change_label = 'Daily % Change'
        elif frequency == 'w':
            pct_change = df[value_col].pct_change(fill_method=None)
            change_label = 'Weekly % Change'
        elif frequency == 'm':
            pct_change = df[value_col].pct_change(fill_method=None)
            change_label = 'Monthly % Change'
        else:
            pct_change = df[value_col].pct_change(fill_method=None)
            change_label = '% Change'

        # Remove infinite and NaN values
        pct_change_clean = pct_change.replace([np.inf, -np.inf], np.nan).dropna()

        if len(pct_change_clean) > 0:
            print(f"\n{change_label} statistics:")
            print(f"Mean: {pct_change_clean.mean():.4%}")
            print(f"Std Dev: {pct_change_clean.std():.4%}")

            # Plot time series and change distribution
            fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8))

            # Time series plot
            ax1.plot(df.index, df[value_col])
            ax1.set_title(f"{series_id} - Time Series")
            ax1.set_ylabel("Value")
            ax1.grid(True, alpha=0.3)

            # Change distribution
            try:
                ax2.hist(pct_change_clean, bins=50, alpha=0.75, color='green', edgecolor='black')
                ax2.set_title(f"{series_id} - {change_label} Distribution")
                ax2.set_xlabel(change_label)
                ax2.set_ylabel("Frequency")
                ax2.axvline(x=0, color='red', linestyle='--', alpha=0.7)
            except ValueError as e:
                print(f"Warning: Could not create histogram for {series_id}: {str(e)}")
                ax2.text(0.5, 0.5, 'Histogram not available\ndue to data issues',
                         ha='center', va='center', transform=ax2.transAxes)

            plt.tight_layout()
            plt.show()
        else:
            print(f"Warning: No valid {change_label} data available for {series_id}")

    # Combine all FRED data for correlation analysis
    print("\n=== Cross-Series Analysis ===")
    combined_df = pd.DataFrame()

    for series_id, df in dataframes_dict.items():
        # Resample all series to monthly frequency for comparison
        if frequencies_dict[series_id] == 'd':
            resampled = df.resample('M').last()
        elif frequencies_dict[series_id] == 'w':
            resampled = df.resample('M').last()
        else:
            resampled = df

        combined_df[series_id] = resampled[resampled.columns[0]]

    # Calculate correlation matrix with handling for NaN values
    combined_pct_change = combined_df.pct_change(fill_method=None)
    combined_pct_change_clean = combined_pct_change.replace([np.inf, -np.inf], np.nan)
    correlation_matrix = combined_pct_change_clean.corr()

    if not correlation_matrix.empty:
        plt.figure(figsize=(12, 10))
        sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, vmin=-1, vmax=1)
        plt.title("Correlation Matrix of Economic Indicators (Monthly % Changes)")
        plt.tight_layout()
        plt.show()
    else:
        print("Warning: Not enough valid data to create correlation matrix")

eda_fred_data((fred_data[0], fred_data[1]))


## Calendar EDA

In [ ]:
def eda_calendar_data(calendar_df):
    """
    Perform EDA on economic calendar data

    Parameters:
    -----------
    calendar_df : pandas.DataFrame
        DataFrame containing economic calendar data
    """
    print("=== EDA for Economic Calendar ===\n")

    # Basic info
    print(f"Date range: {calendar_df['date'].min()} to {calendar_df['date'].max()}")
    print(f"Total number of events: {len(calendar_df)}")

    # Convert date column to datetime - handle potential type issues
    if calendar_df['date'].dtype != 'datetime64[ns]':
        try:
            # Try converting to string first if necessary
            calendar_df['date'] = calendar_df['date'].astype(str)
            calendar_df['date'] = pd.to_datetime(calendar_df['date'], format='%d/%m/%Y')
        except Exception as e:
            print(f"Warning: Could not convert date column: {e}")
            # Try alternative conversion
            try:
                calendar_df['date'] = pd.to_datetime(calendar_df['date'])
            except Exception as e2:
                print(f"Error: Unable to convert date column: {e2}")
                return

    # Extract year and month for analysis
    calendar_df['year'] = calendar_df['date'].dt.year
    calendar_df['month'] = calendar_df['date'].dt.month
    calendar_df['weekday'] = calendar_df['date'].dt.dayofweek

    # Events by type
    print("\n--- Event Categories ---")
    event_types = calendar_df['event'].str.extract(r'(.+?)\s*(?:\(|\s*$)')[0].value_counts()
    print(event_types.head(15))

    # Events by year
    plt.figure(figsize=(12, 6))
    yearly_events = calendar_df.groupby('year').size()
    yearly_events.plot(kind='bar', alpha=0.75, color='blue', edgecolor='black')
    plt.title("Number of Economic Events by Year")
    plt.xlabel("Year")
    plt.ylabel("Number of Events")
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

    # Events by month
    plt.figure(figsize=(12, 6))
    monthly_events = calendar_df.groupby('month').size()
    monthly_events.plot(kind='bar', alpha=0.75, color='green', edgecolor='black')
    plt.title("Number of Economic Events by Month")
    plt.xlabel("Month")
    plt.ylabel("Number of Events")
    plt.xticks(range(12), ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
                           'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

    # Events by weekday
    plt.figure(figsize=(12, 6))
    weekday_events = calendar_df.groupby('weekday').size()
    weekday_events.plot(kind='bar', alpha=0.75, color='orange', edgecolor='black')
    plt.title("Number of Economic Events by Weekday")
    plt.xlabel("Weekday")
    plt.ylabel("Number of Events")
    plt.xticks(range(7), ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'])
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

    # Time of day analysis
    try:
        calendar_df['hour'] = pd.to_datetime(calendar_df['time'].astype(str), format='%H:%M').dt.hour
        plt.figure(figsize=(12, 6))
        hourly_events = calendar_df.groupby('hour').size()
        hourly_events.plot(kind='bar', alpha=0.75, color='purple', edgecolor='black')
        plt.title("Number of Economic Events by Hour of Day")
        plt.xlabel("Hour")
        plt.ylabel("Number of Events")
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()
    except Exception as e:
        print(f"Warning: Could not analyze time of day: {e}")

    # # Event importance
    # print("\n--- Event Importance ---")
    # importance_counts = calendar_df['importance'].value_counts()
    # print(importance_counts)

    # Create a heatmap of events by month and year
    pivot_table = calendar_df.pivot_table(
        values='id',
        index='year',
        columns='month',
        aggfunc='count',
        fill_value=0
    )

    plt.figure(figsize=(12, 8))
    sns.heatmap(pivot_table, cmap='YlOrRd', annot=True, fmt='d')
    plt.title("Event Count Heatmap by Year and Month")
    plt.xlabel("Month")
    plt.ylabel("Year")
    plt.tight_layout()
    plt.show()

    # Most common event types over time
    calendar_df['event_type'] = calendar_df['event'].str.extract(r'(.+?)\s*(?:\(|\s*$)')[0]
    top_5_events = event_types.head(5).index

    plt.figure(figsize=(14, 8))
    for event in top_5_events:
        event_data = calendar_df[calendar_df['event_type'] == event]
        event_by_year = event_data.groupby('year').size()
        plt.plot(event_by_year.index, event_by_year.values, marker='o', label=event)

    plt.title("Top 5 Economic Event Types by Year")
    plt.xlabel("Year")
    plt.ylabel("Count")
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
eda_calendar_data(calendar_df)

# Feature Engineering

## Exploring Trend or Oscillate

In [ ]:
# First, let's access the SPY data from your ticker_data dictionary
spy_df = ticker_data['SPY (US)'].copy()

# Filter to start from the first full market week of 2007
# The first trading day of 2007 was January 3rd (Wednesday)
# So the first full market week started on January 8th (Monday)
start_date = '2007-01-08'
spy_df_filtered = spy_df[spy_df.index >= start_date]

# Calculate the percent change from open to close
spy_df_filtered['pct_change_open_close'] = (spy_df_filtered['SPY_Close'] - spy_df_filtered['SPY_Open']) / spy_df_filtered['SPY_Open'] * 100

# Create the target variable oscillate_i
# oscillate_i = 1 if absolute percent change > 0.5%, else 0
spy_df_filtered['oscillate_i'] = np.where(np.abs(spy_df_filtered['pct_change_open_close']) <= 0.5, 1, 0)

# Display some summary statistics
print(f"Date range: {spy_df_filtered.index.min().date()} to {spy_df_filtered.index.max().date()}")
print(f"Total trading days: {len(spy_df_filtered)}")
print(f"Days with oscillate_i (oscillate_i = 1): {spy_df_filtered['oscillate_i'].sum()}")
print(f"Days without oscillate_i (oscillate_i = 0): {len(spy_df_filtered) - spy_df_filtered['oscillate_i'].sum()}")
print(f"Percentage of oscillate_i days: {spy_df_filtered['oscillate_i'].mean() * 100:.2f}%")

# Let's create a visualization to understand the distribution
import matplotlib.pyplot as plt

# Plot the distribution of daily percentage changes
plt.figure(figsize=(12, 6))
plt.hist(spy_df_filtered['pct_change_open_close'], bins=100, alpha=0.75, edgecolor='black')
plt.axvline(x=0.5, color='red', linestyle='--', linewidth=2, label='0.5% threshold')
plt.axvline(x=-0.5, color='red', linestyle='--', linewidth=2)
plt.xlabel('Daily % Change (Open to Close)')
plt.ylabel('Frequency')
plt.title('Distribution of SPY Daily Percentage Changes (2007-Present)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Create a small sample output to verify the calculation
print("\nSample of the data with oscillate_i:")
sample_data = spy_df_filtered[['SPY_Open', 'SPY_Close', 'pct_change_open_close', 'oscillate_i']].copy()
sample_data['abs_pct_change'] = np.abs(sample_data['pct_change_open_close'])
print(sample_data.head(10))

# Let's also check for balance between trending up and trending down
trending_days = spy_df_filtered[spy_df_filtered['oscillate_i'] == 1]
trend_up = trending_days[trending_days['pct_change_open_close'] > 0]
trend_down = trending_days[trending_days['pct_change_open_close'] < 0]

print(f"\nTrending days (|change| > 0.5%): {len(trending_days)}")
print(f"  Upward trends (change > 0.5%): {len(trend_up)}")
print(f"  Downward trends (change < -0.5%): {len(trend_down)}")

# Let's also check for any outliers or extreme values
print(f"\nMaximum daily % change: {spy_df_filtered['pct_change_open_close'].max():.2f}%")
print(f"Minimum daily % change: {spy_df_filtered['pct_change_open_close'].min():.2f}%")

# Create your base DataFrame for feature engineering
base_df = spy_df_filtered.copy()
print(f"\nBase DataFrame shape: {base_df.shape}")
print(f"Columns: {base_df.columns.tolist()}")

## Building Indice Features

In [ ]:
# First, let's create a list of indices to include (excluding VIX Brazil)
indices_to_include = [key for key in ticker_data.keys() if key != 'VIX Brazil']

# Let's create a function to extract and rename the relevant columns
def extract_columns(df, display_name):
    # Extract ticker symbol from the column names
    ticker_symbol = df.columns[0].split('_')[0]

    # Extract relevant columns and rename them
    columns_to_extract = {}

    if f'{ticker_symbol}_Open' in df.columns:
        columns_to_extract[f'{ticker_symbol}_Open'] = f'{display_name}_Open'
    if f'{ticker_symbol}_High' in df.columns:
        columns_to_extract[f'{ticker_symbol}_High'] = f'{display_name}_High'
    if f'{ticker_symbol}_Low' in df.columns:
        columns_to_extract[f'{ticker_symbol}_Low'] = f'{display_name}_Low'
    if f'{ticker_symbol}_Close' in df.columns:
        columns_to_extract[f'{ticker_symbol}_Close'] = f'{display_name}_Close'
    if f'{ticker_symbol}_Volume' in df.columns:
        columns_to_extract[f'{ticker_symbol}_Volume'] = f'{display_name}_Volume'

    # Create a new dataframe with only the relevant columns
    extracted_df = df[list(columns_to_extract.keys())].copy()
    extracted_df = extracted_df.rename(columns=columns_to_extract)

    return extracted_df

# Join the data from other indices to the base dataframe
for display_name in indices_to_include:
    if display_name != 'SPY (US)':  # We already have SPY in the base_df
        index_df = ticker_data[display_name]

        # Filter to match the date range of base_df
        index_df_filtered = index_df[index_df.index >= start_date]

        # Extract the relevant columns
        extracted_df = extract_columns(index_df_filtered, display_name)

        # Join to base_df
        base_df = base_df.join(extracted_df, how='left')

# Display the resulting dataframe structure
print(f"Base DataFrame shape after joining indices: {base_df.shape}")
print(f"\nColumns in base_df:")
for col in base_df.columns:
    print(f"  {col}")

# Check for missing values in the joined data
missing_summary = base_df.isnull().sum()
if missing_summary.any():
    print("\nMissing values in joined data:")
    print(missing_summary[missing_summary > 0])

# Sample of the data to verify the join
print("\nSample of the joined data:")
sample_columns = ['SPY_Open', 'SPY_High', 'SPY_Low', 'SPY_Close', 'pct_change_open_close', 'oscillate_i']
# Add some other index columns to the sample
for display_name in indices_to_include[:3]:  # Show first 3 indices as example
    if display_name != 'SPY (US)':
        open_col = f'{display_name}_Open'
        high_col = f'{display_name}_High'
        if open_col in base_df.columns and high_col in base_df.columns:
            sample_columns.extend([open_col, high_col])

print(base_df[sample_columns].head())

# Summary of data availability for each index
print("\nData availability summary:")
for display_name in indices_to_include:
    if display_name != 'SPY (US)':
        open_col = f'{display_name}_Open'
        high_col = f'{display_name}_High'
        low_col = f'{display_name}_Low'
        if open_col in base_df.columns:
            non_null_count = base_df[open_col].count()
            total_rows = len(base_df)
            coverage = (non_null_count / total_rows) * 100

            # Check if we have High and Low columns
            has_high = high_col in base_df.columns
            has_low = low_col in base_df.columns

            print(f"{display_name}: {non_null_count}/{total_rows} ({coverage:.1f}% coverage)")
            print(f"  Has High: {has_high}, Has Low: {has_low}")

In [ ]:
def calculate_all_technical_indicators(df):
    """
    Calculate technical indicators for all columns in the dataframe.
    Returns a new dataframe with the original data plus technical indicators.
    """
    df_with_indicators = df.copy()

    # Get all unique ticker names from the columns
    tickers = set()
    for col in df.columns:
        if '_' in col:
            ticker = col.rsplit('_', 1)[0]  # Get everything before the last underscore
            tickers.add(ticker)

    # Remove some non-ticker columns that might have been picked up
    tickers -= {'pct_change_open', 'trend'}

    for ticker in tickers:
        # Check if we have the necessary columns for technical analysis
        has_ohlc = all([
            f'{ticker}_Open' in df.columns,
            f'{ticker}_High' in df.columns,
            f'{ticker}_Low' in df.columns,
            f'{ticker}_Close' in df.columns
        ])

        if has_ohlc:
            # RSI (14-day)
            df_with_indicators[f'{ticker}_RSI_14'] = ta.momentum.RSIIndicator(
                close=df[f'{ticker}_Close'],
                window=14
            ).rsi()

            # ATR (14-day)
            df_with_indicators[f'{ticker}_ATR_14'] = ta.volatility.AverageTrueRange(
                high=df[f'{ticker}_High'],
                low=df[f'{ticker}_Low'],
                close=df[f'{ticker}_Close'],
                window=14
            ).average_true_range()

            # Bollinger Bands
            bollinger = ta.volatility.BollingerBands(
                close=df[f'{ticker}_Close'],
                window=20,
                window_dev=2
            )
            df_with_indicators[f'{ticker}_BB_Upper'] = bollinger.bollinger_hband()
            df_with_indicators[f'{ticker}_BB_Lower'] = bollinger.bollinger_lband()
            df_with_indicators[f'{ticker}_BB_Middle'] = bollinger.bollinger_mavg()

            # MACD
            macd = ta.trend.MACD(close=df[f'{ticker}_Close'])
            df_with_indicators[f'{ticker}_MACD'] = macd.macd()
            df_with_indicators[f'{ticker}_MACD_Signal'] = macd.macd_signal()
            df_with_indicators[f'{ticker}_MACD_Diff'] = macd.macd_diff()

            # Moving Averages
            df_with_indicators[f'{ticker}_SMA_20'] = ta.trend.SMAIndicator(
                close=df[f'{ticker}_Close'],
                window=20
            ).sma_indicator()

            df_with_indicators[f'{ticker}_EMA_20'] = ta.trend.EMAIndicator(
                close=df[f'{ticker}_Close'],
                window=20
            ).ema_indicator()

            # Momentum
            df_with_indicators[f'{ticker}_Momentum_10'] = ta.momentum.ROCIndicator(
                close=df[f'{ticker}_Close'],
                window=10
            ).roc()

            # Stochastic Oscillator
            stoch = ta.momentum.StochasticOscillator(
                high=df[f'{ticker}_High'],
                low=df[f'{ticker}_Low'],
                close=df[f'{ticker}_Close'],
                window=14,
                smooth_window=3
            )
            df_with_indicators[f'{ticker}_Stoch_K'] = stoch.stoch()
            df_with_indicators[f'{ticker}_Stoch_D'] = stoch.stoch_signal()

            # For volume-based indicators (if volume exists)
            if f'{ticker}_Volume' in df.columns:
                # On-Balance Volume
                df_with_indicators[f'{ticker}_OBV'] = ta.volume.OnBalanceVolumeIndicator(
                    close=df[f'{ticker}_Close'],
                    volume=df[f'{ticker}_Volume']
                ).on_balance_volume()

                # Money Flow Index
                df_with_indicators[f'{ticker}_MFI'] = ta.volume.MFIIndicator(
                    high=df[f'{ticker}_High'],
                    low=df[f'{ticker}_Low'],
                    close=df[f'{ticker}_Close'],
                    volume=df[f'{ticker}_Volume'],
                    window=14
                ).money_flow_index()

    return df_with_indicators

# Apply the function to your data
base_df_with_indicators = calculate_all_technical_indicators(base_df)

# Check what indicators were created
print("Technical indicators created for each ticker:")
new_cols = [col for col in base_df_with_indicators.columns if col not in base_df.columns]
print(f"Total new columns: {len(new_cols)}\n")

# Show indicators by ticker
for ticker in sorted(set([col.rsplit('_', 2)[0] for col in new_cols])):
    ticker_indicators = [col for col in new_cols if col.startswith(ticker)]
    if ticker_indicators:
        print(f"{ticker}:")
        for col in ticker_indicators:
            print(f"  - {col}")
        print()

# Verify data integrity
missing_summary = base_df_with_indicators.isnull().sum()
if missing_summary.any():
    print("\nWarning: Some indicators have missing values")
    missing_indicators = missing_summary[missing_summary > 0]
    print(f"Total indicators with missing values: {len(missing_indicators)}")

    # Note: Technical indicators typically have some missing values at the beginning
    # due to their calculation windows (e.g., 14-day RSI will have 13 missing values)
    print("\nMissing values per indicator (first few):")
    for col in missing_indicators.index[:10]:
        print(f"  {col}: {missing_indicators[col]} missing values")

In [ ]:
base_df_with_indicators.columns

## Building Calendar Date Features

In [ ]:
def create_event_indicators(base_df, calendar_df):
    """
    Create binary indicators for economic events with clearer naming convention
    """
    # Ensure calendar_df has datetime index
    calendar_df['date'] = pd.to_datetime(calendar_df['date'], format='%d/%m/%Y')

    # Create indicators initialized to 0
    base_df['cpi'] = 0
    base_df['employment'] = 0
    base_df['fed_meeting'] = 0
    base_df['fed_proj'] = 0

    # Process each event in the calendar
    for _, event in calendar_df.iterrows():
        event_date = event['date'].date()
        event_name = event['event']

        # Skip if the date is not in our base dataframe
        if event_date not in base_df.index.date:
            continue

        # Find the matching index in base_df
        matching_dates = base_df.index.date == event_date
        if not any(matching_dates):
            continue

        date_idx = base_df.index[matching_dates][0]

        # CPI Indicator
        if ('CPI (MoM)' in event_name or 'CPI (YoY)' in event_name) and 'Core' not in event_name:
            base_df.loc[date_idx, 'cpi'] = 1

        # Employment Indicator
        elif 'Nonfarm Payrolls' in event_name or 'Unemployment Rate' in event_name:
            base_df.loc[date_idx, 'employment'] = 1

        # Fed Meeting Indicator
        elif 'Fed Interest Rate Decision' in event_name or \
             'FOMC Statement' in event_name or \
             'FOMC Meeting Minutes' in event_name:
            base_df.loc[date_idx, 'fed_meeting'] = 1

        # Fed Projections Indicator
        elif 'FOMC Economic Projections' in event_name:
            base_df.loc[date_idx, 'fed_proj'] = 1
            # Also set fed_meeting since projections occur during Fed meetings
            base_df.loc[date_idx, 'fed_meeting'] = 1

    # Create lagged and lead versions with clearer naming
    base_df['fed_proj_lag2'] = base_df['fed_proj'].shift(-2)
    base_df['fed_proj_lag1'] = base_df['fed_proj'].shift(1)
    base_df['fed_proj_lead1'] = base_df['fed_proj'].shift(-1)

    base_df['fed_meeting_lag2'] = base_df['fed_meeting'].shift(-2)
    base_df['fed_meeting_lag1'] = base_df['fed_meeting'].shift(1)
    base_df['fed_meeting_lead1'] = base_df['fed_meeting'].shift(-1)

    base_df['cpi_lag2'] = base_df['cpi'].shift(-2)
    base_df['cpi_lag1'] = base_df['cpi'].shift(1)
    base_df['cpi_lead1'] = base_df['cpi'].shift(-1)

    base_df['employment_lag2'] = base_df['employment'].shift(-2)
    base_df['employment_lag1'] = base_df['employment'].shift(1)
    base_df['employment_lead1'] = base_df['employment'].shift(-1)

    return base_df

# Apply the function
base_df_with_indicators = create_event_indicators(base_df_with_indicators, calendar_df)

# Verify the indicators were created
print("Event indicators summary:")
print(f"Days with CPI releases: {base_df_with_indicators['cpi'].sum()}")
print(f"Days with Employment releases: {base_df_with_indicators['employment'].sum()}")
print(f"Days with Fed meetings: {base_df_with_indicators['fed_meeting'].sum()}")
print(f"Days with Fed projections: {base_df_with_indicators['fed_proj'].sum()}")

# Let's verify by checking specific dates around a known Fed projection event
# For the 2023-12-13 FOMC Economic Projections
test_dates = pd.date_range(start='2023-12-11', end='2023-12-15')
test_dates = [d for d in test_dates if d in base_df_with_indicators.index]

if test_dates:
    test_df = base_df_with_indicators.loc[test_dates]
    print("\nFed projection indicators around December 13, 2023:")
    print(test_df[['fed_proj', 'fed_proj_lag1', 'fed_proj_lag2', 'fed_proj_lead1']])
else:
    print("\nNo matching dates found in base_df for the test period")

In [ ]:
asian_markets = ['Nikkei 225 (Japan)', 'Hang Seng (Hong Kong)', 'SSE Composite (China)', 'ASX 200 (Australia)']
european_markets = ['DAX (Germany)', 'FTSE 100 (UK)', 'CAC 40 (France)', 'Euro Stoxx 50 (EU)']
us_markets = ['SPY']
currency_pairs = ['EUR/USD', 'JPY/USD', 'CNY/USD']
commodities = ['Gold', 'Crude Oil', 'Silver', 'Corn', 'Copper']
volatility_indices = ['VIX (US)', 'US Dollar Index']

event_indicators = [
    # Base indicators
    'cpi', 'employment', 'fed_meeting', 'fed_proj',

    # Lagged indicators (past events)
    'cpi_lag1', 'employment_lag1', 'fed_meeting_lag1', 'fed_proj_lag1',
    'cpi_lag2', 'employment_lag2', 'fed_meeting_lag2', 'fed_proj_lag2',

    # Lead indicators (future events)
    'cpi_lead1', 'employment_lead1', 'fed_meeting_lead1', 'fed_proj_lead1'
]

def create_training_dataframe(base_df, base_df_with_indicators, fred_data=None, fred_frequencies=None):
    """
    Create a clean dataframe for model training with specific requirements for each market type
    Also handles FRED data with appropriate lags for 10 AM Eastern predictions
    """
    training_df = pd.DataFrame(index=base_df_with_indicators.index)

    # Define market groups
    asian_markets = ['Nikkei 225 (Japan)', 'Hang Seng (Hong Kong)', 'SSE Composite (China)', 'ASX 200 (Australia)']
    european_markets = ['DAX (Germany)', 'FTSE 100 (UK)', 'CAC 40 (France)', 'Euro Stoxx 50 (EU)']
    us_markets = ['SPY']
    currency_pairs = ['EUR/USD', 'JPY/USD', 'CNY/USD']
    commodities = ['Gold', 'Crude Oil', 'Silver', 'Corn', 'Copper']
    volatility_indices = ['VIX (US)', 'US Dollar Index']

    # Define technical indicators to include
    technical_indicators = [
        'RSI_14', 'ATR_14', 'BB_Upper', 'BB_Lower', 'BB_Middle',
        'MACD', 'MACD_Signal', 'MACD_Diff', 'SMA_20', 'EMA_20',
        'Momentum_10', 'Stoch_K', 'Stoch_D', 'OBV', 'MFI'
    ]

    # 1. Asian markets - technical indicators with no lag
    for market in asian_markets:
        for indicator in technical_indicators:
            col_name = f'{market}_{indicator}'
            if col_name in base_df_with_indicators.columns:
                training_df[f'{market}_{indicator}_current'] = base_df_with_indicators[col_name]

    # 2. European markets - lag 1 technical indicators + current day open
    for market in european_markets:
        # Lag 1 technical indicators
        for indicator in technical_indicators:
            col_name = f'{market}_{indicator}'
            if col_name in base_df_with_indicators.columns:
                training_df[f'{market}_{indicator}_lag1'] = base_df_with_indicators[col_name].shift(1)

        # Current day open
        if f'{market}_Open' in base_df.columns:
            training_df[f'{market}_Open_current'] = base_df[f'{market}_Open']

    # 3. SPY - lag 1 technical indicators + current day open + lag 1 raw values
    # Lag 1 technical indicators
    for indicator in technical_indicators:
        col_name = f'SPY_{indicator}'
        if col_name in base_df_with_indicators.columns:
            training_df[f'SPY_{indicator}_lag1'] = base_df_with_indicators[col_name].shift(1)

    # Current day open
    if 'SPY_Open' in base_df.columns:
        training_df['SPY_Open_current'] = base_df['SPY_Open']

    # Lag 1 raw values for SPY
    spy_raw_columns = ['SPY_Close', 'SPY_High', 'SPY_Low', 'SPY_Volume']
    for col in spy_raw_columns:
        if col in base_df.columns:
            training_df[f'{col}_lag1'] = base_df[col].shift(1)

    # 4. Currency pairs, commodities, volatility indices - lag 1 technical indicators + current day open
    combined_markets = currency_pairs + commodities + volatility_indices

    for market in combined_markets:
        # Lag 1 technical indicators
        for indicator in technical_indicators:
            col_name = f'{market}_{indicator}'
            if col_name in base_df_with_indicators.columns:
                training_df[f'{market}_{indicator}_lag1'] = base_df_with_indicators[col_name].shift(1)

        # Current day open
        if f'{market}_Open' in base_df.columns:
            training_df[f'{market}_Open_current'] = base_df[f'{market}_Open']

    # 5. Add economic event indicators
    event_indicators = [
    # Base indicators
    'cpi', 'employment', 'fed_meeting', 'fed_proj',

    # Lagged indicators (past events)
    'cpi_lag1', 'employment_lag1', 'fed_meeting_lag1', 'fed_proj_lag1',
    'cpi_lag2', 'employment_lag2', 'fed_meeting_lag2', 'fed_proj_lag2',

    # Lead indicators (future events)
    'cpi_lead1', 'employment_lead1', 'fed_meeting_lead1', 'fed_proj_lead1'
]

    for indicator in event_indicators:
        if indicator in base_df_with_indicators.columns:
            training_df[indicator] = base_df_with_indicators[indicator]

    # 6. Process FRED data if provided
    if fred_data is not None and fred_frequencies is not None:
        # Define series by their release timing
        daily_series = ['DFF', 'T10Y2Y', 'GS5', 'GS30']  # Usually updated after market close
        weekly_series = ['STLFSI', 'BAMLH0A0HYM2', 'BAMLC0A0CM']  # Updated on specific days
        monthly_series = ['CPIAUCSL', 'UNRATE', 'M2SL', 'USSLIND']  # Fixed monthly releases

        # Create a date range matching base_df
        date_range = pd.date_range(start=base_df.index.min(), end=base_df.index.max(), freq='D')

        for series_id, df in fred_data.items():
            value_col = df.columns[0]
            freq = fred_frequencies[series_id]

            # Reindex to daily frequency first
            df_daily = df.reindex(date_range)

            if series_id in daily_series:
                # For daily data, use 1-day lag to ensure availability at 10 AM
                df_daily[value_col] = df_daily[value_col].ffill().shift(1)

            elif series_id in weekly_series:
                # For weekly data, handle based on release schedule
                if series_id == 'STLFSI':  # St. Louis Fed Financial Stress Index updates Thursday
                    # Use 4-day lag to ensure we have data at 10 AM Monday
                    df_daily[value_col] = df_daily[value_col].ffill().shift(4)
                else:
                    # Other weekly series: use 1-day lag
                    df_daily[value_col] = df_daily[value_col].ffill().shift(1)

            elif series_id in monthly_series:
                # For monthly data, forward fill and apply appropriate lag
                df_daily[value_col] = df_daily[value_col].ffill()

                if series_id == 'CPIAUCSL':  # CPI releases around the 10th-15th
                    df_daily[value_col] = df_daily[value_col].shift(15)
                elif series_id == 'UNRATE':  # Unemployment rate releases first Friday
                    df_daily[value_col] = df_daily[value_col].shift(10)
                else:  # M2 and Leading Index
                    df_daily[value_col] = df_daily[value_col].shift(20)

            # Merge with training_df
            training_df[f'{series_id}_10am'] = df_daily[value_col]

            # Add derived features for FRED data
            if series_id == 'DFF':
                training_df['fed_funds_change'] = training_df[f'{series_id}_10am'].pct_change()
            elif series_id == 'T10Y2Y':
                training_df['yield_curve_change'] = training_df[f'{series_id}_10am'].diff()
            elif series_id == 'STLFSI':
                training_df['financial_stress_change'] = training_df[f'{series_id}_10am'].diff()
            elif series_id == 'CPIAUCSL':
                training_df['cpi_mom'] = training_df[f'{series_id}_10am'].pct_change()
            elif series_id == 'UNRATE':
                training_df['unemployment_change'] = training_df[f'{series_id}_10am'].diff()

        # Add interaction features for FRED data
        if 'T10Y2Y_10am' in training_df.columns and 'STLFSI_10am' in training_df.columns:
            training_df['yield_curve_stress'] = training_df['T10Y2Y_10am'] * training_df['STLFSI_10am']

    # 7. Add target variable
    if 'oscillate_i' in base_df.columns:
        training_df['oscillate_i_current'] = base_df['oscillate_i']

    # Drop the first 33 rows to account for:
    # - SMA/EMA 20-day calculations (20 days)
    # - RSI/ATR 14-day calculations (14 days)
    # - MACD calculations (26 days for standard MACD)
    # - 1 additional row for lag operations
    # - A few extra rows for safety
    training_df = training_df.iloc[33:].copy()

    # Handle any remaining NaN values at the beginning of the series for FRED data
    for col in training_df.columns:
        if '_10am' in col:
            # For the beginning of the series where we don't have enough history
            training_df[col] = training_df[col].bfill()

    # Verify no null values remain
    null_count = training_df.isnull().sum().sum()
    print(f"Total null values after dropping first 33 rows: {null_count}")

    return training_df

# Create the training dataframe with FRED data and event indicators
training_df = create_training_dataframe(base_df, base_df_with_indicators, fred_data[0], fred_data[1])

# Display structure and verify
print(f"Training dataframe shape: {training_df.shape}")
print("\nColumn types by market group:")

# Count columns by type
asian_cols = [col for col in training_df.columns if any(market in col for market in asian_markets)]
euro_cols = [col for col in training_df.columns if any(market in col for market in european_markets)]
spy_cols = [col for col in training_df.columns if 'SPY' in col]
currency_cols = [col for col in training_df.columns if any(pair in col for pair in currency_pairs)]
commodity_cols = [col for col in training_df.columns if any(comm in col for comm in commodities)]
volatility_cols = [col for col in training_df.columns if any(vol in col for vol in volatility_indices)]
fred_cols = [col for col in training_df.columns if '_10am' in col or col in ['fed_funds_change', 'yield_curve_change', 'financial_stress_change', 'cpi_mom', 'unemployment_change', 'yield_curve_stress']]
event_cols = [col for col in training_df.columns if col in event_indicators]

print(f"Asian market columns: {len(asian_cols)}")
print(f"European market columns: {len(euro_cols)}")
print(f"SPY columns: {len(spy_cols)}")
print(f"Currency pair columns: {len(currency_cols)}")
print(f"Commodity columns: {len(commodity_cols)}")
print(f"Volatility index columns: {len(volatility_cols)}")
print(f"FRED economic indicator columns: {len(fred_cols)}")
print(f"Economic event indicator columns: {len(event_cols)}")

# Final check for any remaining missing values
missing_summary = training_df.isnull().sum()
if missing_summary.any():
    print("\nWarning: Some columns still have missing values")
    missing_df = pd.DataFrame({'Missing_Count': missing_summary[missing_summary > 0]})
    missing_df['Percentage'] = (missing_df['Missing_Count'] / len(training_df) * 100).round(2)
    print(f"Total columns with missing values: {len(missing_df)}")

    # Find columns with more than 50% missing values
    high_missing_cols = missing_df[missing_df['Percentage'] > 50].index.tolist()

    if high_missing_cols:
        print(f"\nDropping {len(high_missing_cols)} columns with >50% missing values:")
        for col in high_missing_cols:
            print(f"  - {col}: {missing_df.loc[col, 'Percentage']}% missing")

        # Drop these columns
        training_df = training_df.drop(columns=high_missing_cols)

        # Recalculate missing summary after dropping columns
        missing_summary = training_df.isnull().sum()
        missing_df = pd.DataFrame({'Missing_Count': missing_summary[missing_summary > 0]})
        missing_df['Percentage'] = (missing_df['Missing_Count'] / len(training_df) * 100).round(2)

    print("\nRemaining columns with missing values (if any):")
    if len(missing_df) > 0:
        print(missing_df.head(10))
    else:
        print("No more columns with missing values!")
else:
    print("\nSuccess: No missing values in the training dataframe!")

# Sample of the final dataframe
print("\nFirst 5 rows of training dataframe:")
print(training_df.head())

# Print final shape
print(f"\nFinal training dataframe shape: {training_df.shape}")

# Model Development

## Train Test split and building Evaluation Function

In [ ]:
# Cell 3: Filter data to keep only 2007-2024 (drop 2025)
# Filter out 2025 data
base_df_filtered = training_df[training_df.index.year < 2025].copy()

# Split into train (2007-2023) and test (2024)
train_data = base_df_filtered[base_df_filtered.index.year < 2024]
test_data = base_df_filtered[base_df_filtered.index.year == 2024]

print(f"Train data period: {train_data.index.min().date()} to {train_data.index.max().date()}")
print(f"Test data period: {test_data.index.min().date()} to {test_data.index.max().date()}")
print(f"Train shape: {train_data.shape}, Test shape: {test_data.shape}")


In [ ]:
def prepare_data(train_df, test_df):
    """
    Prepare the data for modeling by handling missing values and creating X, y
    """
    # Separate features and target for train
    X_train = train_df.drop('oscillate_i_current', axis=1)
    y_train = train_df['oscillate_i_current']

    # Separate features and target for test
    X_test = test_df.drop('oscillate_i_current', axis=1)
    y_test = test_df['oscillate_i_current']

    # Handle missing values using imputer fitted on training data
    imputer = SimpleImputer(strategy='mean')
    X_train_imputed = pd.DataFrame(imputer.fit_transform(X_train),
                                   columns=X_train.columns,
                                   index=X_train.index)
    X_test_imputed = pd.DataFrame(imputer.transform(X_test),
                                  columns=X_test.columns,
                                  index=X_test.index)

    # Scale features using scaler fitted on training data
    scaler = StandardScaler()
    X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train_imputed),
                                  columns=X_train.columns,
                                  index=X_train.index)
    X_test_scaled = pd.DataFrame(scaler.transform(X_test_imputed),
                                 columns=X_test.columns,
                                 index=X_test.index)

    return X_train_scaled, X_test_scaled, y_train, y_test, scaler, imputer

In [ ]:
X_train, X_test, y_train, y_test, scaler, imputer = prepare_data(train_data, test_data)
print(f"X_train shape: {X_train.shape}, X_test shape: {X_test.shape}")
print(f"y_train distribution:\n{y_train.value_counts()}")
print(f"y_test distribution:\n{y_test.value_counts()}")

In [ ]:
def evaluate_model(y_true, y_pred, model_name):
    """
    Evaluate model performance and visualize results
    """
    print(f"\n{model_name} Performance on 2024 Test Data:")
    print(f"Accuracy: {accuracy_score(y_true, y_pred):.4f}")

    # Calculate naive predictor accuracy
    naive_accuracy = max(y_true.mean(), 1 - y_true.mean())
    improvement = (accuracy_score(y_true, y_pred) - naive_accuracy) * 100
    print(f"Naive Predictor Accuracy: {naive_accuracy:.4f}")
    print(f"Improvement over Naive: {improvement:.2f}%")

    print("\nClassification Report:")
    print(classification_report(y_true, y_pred))

    # Confusion Matrix
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title(f'{model_name} Confusion Matrix - 2024 Test Data')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.show()

    # Return metrics for comparison
    return {
        'accuracy': accuracy_score(y_true, y_pred),
        'improvement': improvement,
        'confusion_matrix': cm,
        'classification_report': classification_report(y_true, y_pred, output_dict=True)
    }


## Feature Selection

In [ ]:
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
import numpy as np
import pandas as pd

def select_important_features(X_train, y_train, X_test,
                              num_features=30, random_state=42, oversample=True):
    """
    1) Optionally SMOTE‐upsample the minority class in X_train/y_train
    2) Fit RandomForest to determine top `num_features`
    3) Return the **oversampled** & **column‐filtered** X_train plus filtered X_test

    Returns
    -------
    X_train_filtered : pd.DataFrame  # may have more rows if oversampled
    X_test_filtered  : pd.DataFrame
    y_train_resampled: pd.Series     # aligned with X_train_filtered
    """
    # 1) Copy and optionally SMOTE
    X_tr, y_tr = X_train.copy(), y_train.copy()
    if oversample:
        sm = SMOTE(random_state=random_state)
        X_res, y_res = sm.fit_resample(X_tr, y_tr)
        # wrap back into DataFrame
        X_tr = pd.DataFrame(X_res, columns=X_train.columns)
        y_tr = pd.Series(y_res, name=y_train.name)

    # 2) Train RF on (possibly oversampled) data
    model = RandomForestClassifier(
        n_estimators=100, random_state=random_state, n_jobs=-1
    )
    model.fit(X_tr, y_tr)

    # 3) Pick top features
    importances = model.feature_importances_
    top_idx     = np.argsort(importances)[::-1][:num_features]
    top_cols    = X_train.columns[top_idx]

    # 4) Subset **oversampled** train and original test
    X_train_filtered = X_tr[top_cols].reset_index(drop=True)
    X_test_filtered  = X_test[top_cols].copy()

    return X_train_filtered, X_test_filtered, y_tr.reset_index(drop=True)


X_train_filt, X_test_filt, y_train_resampled  = select_important_features(
    X_train, y_train, X_test, num_features=150
)
print("Filtered shapes:", X_train_filt.shape, X_test_filt.shape)


## Building XGBoost Classifier

In [ ]:
def train_xgboost(X_train, y_train, X_test, n_splits=5):
    """
    Train XGBoost using time series cross-validation for validation,
    then train final model on all training data
    """
    # Use TimeSeriesSplit on training data for validation
    tscv = TimeSeriesSplit(n_splits=n_splits)
    cv_scores = []

    # Validate model using time series CV
    for fold, (train_idx, val_idx) in enumerate(tscv.split(X_train)):
        X_train_fold = X_train.iloc[train_idx]
        X_val_fold = X_train.iloc[val_idx]
        y_train_fold = y_train.iloc[train_idx]
        y_val_fold = y_train.iloc[val_idx]

        # Create and train model for this fold
        model = XGBClassifier(
            n_estimators=100,
            max_depth=6,
            learning_rate=0.1,
            subsample=0.8,
            colsample_bytree=0.8,
            random_state=42
        )

        model.fit(X_train_fold, y_train_fold)

        # Validate on fold
        y_pred_val = model.predict(X_val_fold)
        fold_score = accuracy_score(y_val_fold, y_pred_val)
        cv_scores.append(fold_score)

        print(f"Fold {fold + 1}/{n_splits} validation accuracy: {fold_score:.4f}")

    print(f"Average CV accuracy: {np.mean(cv_scores):.4f} (+/- {np.std(cv_scores):.4f})")

    # Train final model on all training data
    final_model = XGBClassifier(
        n_estimators=100,
        max_depth=6,
        learning_rate=0.1,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42
    )

    final_model.fit(X_train, y_train)

    # Make predictions on test data
    y_pred = final_model.predict(X_test)

    return y_pred, final_model, cv_scores

# Train and evaluate XGBoost
y_pred_xgb, model_xgb, xgb_cv_scores = train_xgboost(X_train_filt, y_train_resampled, X_test_filt)
xgb_results = evaluate_model(y_test, y_pred_xgb, 'XGBoost')

## Build Logistic Regression

In [ ]:
def train_logistic_regression(X_train, y_train, X_test, n_splits=5):
    """
    Train Logistic Regression using time series cross-validation for validation,
    then train final model on all training data
    """
    # Use TimeSeriesSplit on training data for validation
    tscv = TimeSeriesSplit(n_splits=n_splits)
    cv_scores = []

    # Validate model using time series CV
    for fold, (train_idx, val_idx) in enumerate(tscv.split(X_train)):
        X_train_fold = X_train.iloc[train_idx]
        X_val_fold = X_train.iloc[val_idx]
        y_train_fold = y_train.iloc[train_idx]
        y_val_fold = y_train.iloc[val_idx]

        # Create and train model for this fold
        model = LogisticRegression(
            random_state=42,
            max_iter=1000,
            class_weight='balanced'
        )

        model.fit(X_train_fold, y_train_fold)

        # Validate on fold
        y_pred_val = model.predict(X_val_fold)
        fold_score = accuracy_score(y_val_fold, y_pred_val)
        cv_scores.append(fold_score)

        print(f"Fold {fold + 1}/{n_splits} validation accuracy: {fold_score:.4f}")

    print(f"Average CV accuracy: {np.mean(cv_scores):.4f} (+/- {np.std(cv_scores):.4f})")

    # Train final model on all training data
    final_model = LogisticRegression(
        random_state=42,
        max_iter=1000,
        class_weight='balanced'
    )

    final_model.fit(X_train, y_train)

    # Make predictions on test data
    y_pred = final_model.predict(X_test)

    return y_pred, final_model, cv_scores

# Train and evaluate Logistic Regression
y_pred_lr, model_lr, lr_cv_scores = train_logistic_regression(X_train_filt, y_train_resampled, X_test_filt)
lr_results = evaluate_model(y_test, y_pred_lr, 'Logistic Regression')


## Build LSTM Neural Network

In [ ]:
from sklearn.model_selection import train_test_split  # Add this import

def build_lstm_model(hp):
    """
    Create an LSTM model with hyperparameters to tune
    """
    model = tf.keras.Sequential([
        Input(shape=(1, X_train_filt.shape[1])),

        # LSTM layers with tunable units
        LSTM(hp.Int('lstm_units_1', min_value=32, max_value=96, step=32),
             return_sequences=True),
        Dropout(hp.Float('dropout_1', min_value=0.2, max_value=0.5, step=0.1)),

        LSTM(hp.Int('lstm_units_2', min_value=16, max_value=48, step=16)),
        Dropout(hp.Float('dropout_2', min_value=0.2, max_value=0.5, step=0.1)),

        # Dense layer with tunable units
        Dense(hp.Int('dense_units', min_value=8, max_value=32, step=8),
              activation='relu'),
        Dropout(hp.Float('dropout_3', min_value=0.2, max_value=0.5, step=0.1)),

        Dense(1, activation='sigmoid')
    ])

    # Compile with tunable learning rate
    model.compile(
        optimizer=Adam(hp.Float('learning_rate',
                               min_value=1e-4,
                               max_value=1e-2,
                               sampling='LOG')),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    return model

def tune_lstm_hyperparameters(X_train, y_train, max_trials=10, epochs=20, batch_size=32):
    """
    Tune LSTM hyperparameters using Keras Tuner
    """
    # Reshape data for LSTM
    X_train_lstm = X_train.values.reshape(X_train.shape[0], 1, X_train.shape[1])

    # Compute class weights
    class_weights = compute_class_weight('balanced',
                                       classes=np.unique(y_train),
                                       y=y_train)
    class_weight_dict = dict(enumerate(class_weights))

    # Split data for validation
    X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(
        X_train_lstm, y_train, test_size=0.2, shuffle=False
    )

    # Create tuner - Using Hyperband for efficiency on limited resources
    tuner = kt.Hyperband(
        build_lstm_model,
        objective='val_accuracy',
        max_epochs=epochs,
        factor=3,
        directory='lstm_tuning',
        project_name='lstm_hyperopt'
    )

    # Callbacks
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

    # Search for best hyperparameters
    tuner.search(
        X_train_split,
        y_train_split,
        validation_data=(X_val_split, y_val_split),
        epochs=epochs,
        batch_size=batch_size,
        callbacks=[early_stopping],
        class_weight=class_weight_dict,
        verbose=1
    )

    # Get best hyperparameters
    best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

    print("\nBest hyperparameters:")
    print(f"LSTM units 1: {best_hps.get('lstm_units_1')}")
    print(f"LSTM units 2: {best_hps.get('lstm_units_2')}")
    print(f"Dense units: {best_hps.get('dense_units')}")
    print(f"Dropout 1: {best_hps.get('dropout_1')}")
    print(f"Dropout 2: {best_hps.get('dropout_2')}")
    print(f"Dropout 3: {best_hps.get('dropout_3')}")
    print(f"Learning rate: {best_hps.get('learning_rate')}")

    return best_hps, tuner

# Modified train_lstm function to use best hyperparameters
def train_optimized_lstm(X_train, y_train, X_test, best_hps, n_splits=5, epochs=30, batch_size=32):
    """
    Train LSTM using the best hyperparameters found
    """
    # Use TimeSeriesSplit on training data for validation
    tscv = TimeSeriesSplit(n_splits=n_splits)
    cv_scores = []

    # Reshape data for LSTM
    X_train_lstm = X_train.values.reshape(X_train.shape[0], 1, X_train.shape[1])
    X_test_lstm = X_test.values.reshape(X_test.shape[0], 1, X_test.shape[1])

    # Compute class weights
    class_weights = compute_class_weight('balanced',
                                       classes=np.unique(y_train),
                                       y=y_train)
    class_weight_dict = dict(enumerate(class_weights))

    # Create final model with best hyperparameters
    model = tf.keras.Sequential([
        Input(shape=(1, X_train.shape[1])),
        LSTM(best_hps.get('lstm_units_1'), return_sequences=True),
        Dropout(best_hps.get('dropout_1')),
        LSTM(best_hps.get('lstm_units_2')),
        Dropout(best_hps.get('dropout_2')),
        Dense(best_hps.get('dense_units'), activation='relu'),
        Dropout(best_hps.get('dropout_3')),
        Dense(1, activation='sigmoid')
    ])

    model.compile(optimizer=Adam(best_hps.get('learning_rate')),
                 loss='binary_crossentropy',
                 metrics=['accuracy'])

    # Callbacks
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.0001)

    # Train final model
    history = model.fit(
        X_train_lstm, y_train,
        validation_split=0.2,
        epochs=epochs,
        batch_size=batch_size,
        callbacks=[early_stopping, reduce_lr],
        class_weight=class_weight_dict,
        verbose=1
    )

    # Make predictions
    y_pred_proba = model.predict(X_test_lstm, verbose=0)

    # Find optimal threshold
    precision, recall, thresholds = precision_recall_curve(y_test, y_pred_proba)
    f1_scores = 2 * (precision * recall) / (precision + recall + 1e-10)
    optimal_threshold = thresholds[np.argmax(f1_scores)]

    y_pred = (y_pred_proba > optimal_threshold).astype(int).flatten()

    return y_pred, model, cv_scores

# Main execution
print("Starting hyperparameter tuning...")
best_hps, tuner = tune_lstm_hyperparameters(X_train_filt, y_train_resampled,
                                           max_trials=10, epochs=20)

print("\nTraining LSTM with optimized hyperparameters...")
y_pred_lstm, model_lstm, lstm_cv_scores = train_optimized_lstm(
   X_train_filt, y_train_resampled, X_test_filt, best_hps
)

lstm_results = evaluate_model(y_test, y_pred_lstm, 'Optimized LSTM')

# Model Evaluation

In [ ]:
# Cell 10: Compare all models
def compare_models(results_dict):
    """
    Compare the performance of all models
    """
    model_names = list(results_dict.keys())
    accuracies = [results_dict[model]['accuracy'] for model in model_names]
    improvements = [results_dict[model]['improvement'] for model in model_names]

    # Create comparison chart
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

    # Accuracy comparison
    bars1 = ax1.bar(model_names, accuracies)
    ax1.set_title('Model Comparison - Accuracy on 2024 Test Data')
    ax1.set_ylabel('Accuracy')
    ax1.set_ylim(0, 1)

    # Add value labels on bars
    for bar, accuracy in zip(bars1, accuracies):
        ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                f'{accuracy:.3f}', ha='center', va='bottom')

    # Improvement comparison
    bars2 = ax2.bar(model_names, improvements, color=['green' if imp > 0 else 'red' for imp in improvements])
    ax2.set_title('Model Comparison - Improvement over Naive Predictor')
    ax2.set_ylabel('Improvement (%)')
    ax2.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
    ax2.axhline(y=12, color='blue', linestyle='--', linewidth=2, label='Full Credit Threshold (12%)')
    ax2.legend()

    # Add value labels on bars
    for bar, improvement in zip(bars2, improvements):
        ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5 if improvement > 0 else bar.get_height() - 1,
                f'{improvement:.1f}%', ha='center', va='bottom' if improvement > 0 else 'top')

    plt.tight_layout()
    plt.show()

# Compare all models
results = {
    'Logistic Regression': lr_results,
    'XGBoost': xgb_results,
    'lstm': lstm_results
}

compare_models(results)



In [30]:
# Cell 12: Print summary of results
print("=" * 50)
print("MODEL PERFORMANCE SUMMARY ON 2024 TEST DATA")
print("=" * 50)
for model_name, result in results.items():
    print(f"\n{model_name}:")
    print(f"  Accuracy: {result['accuracy']:.4f}")
    print(f"  Improvement over Naive: {result['improvement']:.2f}%")
    print(f"  Precision (class 1): {result['classification_report']['1']['precision']:.4f}")
    print(f"  Recall (class 1): {result['classification_report']['1']['recall']:.4f}")
    print(f"  F1-score (class 1): {result['classification_report']['1']['f1-score']:.4f}")

    # Check if meets assignment requirements
    if result['improvement'] >= 12:
        print(f"  STATUS: MEETS FULL CREDIT REQUIREMENT (≥12% improvement)")
    else:
        print(f"  STATUS: Needs {12 - result['improvement']:.2f}% more improvement for full credit")

MODEL PERFORMANCE SUMMARY ON 2024 TEST DATA

Logistic Regression:
  Accuracy: 0.6825
  Improvement over Naive: 0.79%
  Precision (class 1): 0.7143
  Recall (class 1): 0.8824
  F1-score (class 1): 0.7895
  STATUS: Needs 11.21% more improvement for full credit

XGBoost:
  Accuracy: 0.6786
  Improvement over Naive: 0.40%
  Precision (class 1): 0.7306
  Recall (class 1): 0.8294
  F1-score (class 1): 0.7769
  STATUS: Needs 11.60% more improvement for full credit

lstm:
  Accuracy: 0.7222
  Improvement over Naive: 4.76%
  Precision (class 1): 0.7358
  Recall (class 1): 0.9176
  F1-score (class 1): 0.8168
  STATUS: Needs 7.24% more improvement for full credit
